In [1]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [14]:
r = requests.get('https://myjsustainagri.com/home/archive/')

In [15]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a')
pages = [a_.get('href') for a_ in a if '/mjsa-' in a_.get('href') or '/mjsa.' in a_.get('href')]
pages = [a_ for a_ in pages if '/wp-content/uploads/' not in a_ and 'doi.org/' not in a_]
pages = sorted(list(set(pages)))
len(pages)

120

In [16]:
pages

['https://myjsustainagri.com/mjsa-01-2019-28-32/',
 'https://myjsustainagri.com/mjsa-01-2019-33-38/',
 'https://myjsustainagri.com/mjsa-01-2019-39-43/',
 'https://myjsustainagri.com/mjsa-01-2019-44-48/',
 'https://myjsustainagri.com/mjsa-01-2020-01-04/',
 'https://myjsustainagri.com/mjsa-01-2020-05-09/',
 'https://myjsustainagri.com/mjsa-01-2020-10-15/',
 'https://myjsustainagri.com/mjsa-01-2020-16-21/',
 'https://myjsustainagri.com/mjsa-01-2020-22-25/',
 'https://myjsustainagri.com/mjsa-01-2020-26-28/',
 'https://myjsustainagri.com/mjsa-01-2020-29-33/',
 'https://myjsustainagri.com/mjsa-01-2020-34-35/',
 'https://myjsustainagri.com/mjsa-01-2020-36-39/',
 'https://myjsustainagri.com/mjsa-01-2020-40-43/',
 'https://myjsustainagri.com/mjsa-01-2021-01-05/',
 'https://myjsustainagri.com/mjsa-01-2021-06-09/',
 'https://myjsustainagri.com/mjsa-01-2021-10-15/',
 'https://myjsustainagri.com/mjsa-01-2021-16-20/',
 'https://myjsustainagri.com/mjsa-01-2021-21-23/',
 'https://myjsustainagri.com/mj

In [19]:
from tqdm import tqdm

urls = []
for p in tqdm(pages):
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if '/download/' in a_.get('href')]
    urls.extend(a)
    
urls

100%|█████████████████████████████████████████| 120/120 [01:26<00:00,  1.38it/s]


['https://myjsustainagri.com/download/13835/',
 'https://myjsustainagri.com/download/13837/',
 'https://myjsustainagri.com/download/13839/',
 'https://myjsustainagri.com/download/13841/',
 'https://myjsustainagri.com/download/13963/',
 'https://myjsustainagri.com/download/14115/',
 'https://myjsustainagri.com/download/14027/',
 'https://myjsustainagri.com/download/13983/',
 'https://myjsustainagri.com/download/14019/',
 'https://myjsustainagri.com/download/13997/',
 'https://myjsustainagri.com/download/13992/',
 'https://myjsustainagri.com/download/14014/',
 'https://myjsustainagri.com/download/13975/',
 'https://myjsustainagri.com/download/14222/',
 'https://myjsustainagri.com/download/14427/',
 'https://myjsustainagri.com/download/14432/',
 'https://myjsustainagri.com/download/14437/',
 'https://myjsustainagri.com/download/14443/',
 'https://myjsustainagri.com/download/14449/',
 'https://myjsustainagri.com/download/14455/',
 'https://myjsustainagri.com/download/14461/',
 'https://myj

In [20]:
urls = sorted(list(set(urls)))
len(urls)

119

In [21]:
!mkdir myjsustainagri.com

In [23]:
import os

pdfs = urls
for i in tqdm(range(len(pdfs))):
    filename = os.path.join('myjsustainagri.com', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i]
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|█████████████████████████████████████████| 119/119 [01:45<00:00,  1.12it/s]


In [24]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [25]:
files = glob('myjsustainagri.com/*.pdf')

with open('myjsustainagri.com.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|█████████████████████████████████████████| 119/119 [00:06<00:00, 17.52it/s]
